In [1]:
%matplotlib notebook

import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [2]:
with open('3dinfo.pickle', 'rb') as f:
    dim3 = pickle.load(f)

In [3]:
with open('flattened.pickle', 'rb') as f:
    flattened = pickle.load(f)

proj, dist = flattened
print(proj.shape)
print(dist.shape)
print(dim3.shape)

(30876, 2)
(30876,)
(30876, 3)


In [4]:
sortIdx = np.argsort(dist)
sortedDist = np.flip(dist[sortIdx], axis=0)
sortedProj = np.flip(proj[sortIdx], axis=0)
sortedDim3 = np.flip(dim3[sortIdx], axis=0)


In [9]:
max_x = (max(sortedProj.T[0]))
max_y = (max(sortedProj.T[1]))
min_x = (min(sortedProj.T[0]))
min_y = (min(sortedProj.T[1]))

max_r = max(sortedDim3.T[0])
max_g = max(sortedDim3.T[1])
max_b = max(sortedDim3.T[2])
min_r = min(sortedDim3.T[0])
min_g = min(sortedDim3.T[1])
min_b = min(sortedDim3.T[2])

print(max_x - min_x)
print(max_y - min_y)

range_y = max_y - min_y
range_x = max_x - min_x

img_asp_rat = range_y / range_x

img_w = 2000
img_h = int(img_w * img_asp_rat)
border_px = 100
print(img_w, img_h)
max_iw = img_w - border_px
max_ih = img_h - border_px

18.811296
16.572609
2000 1761


In [10]:
print(type(dim3[0]))
r, g, b = dim3[0]
print(r, g, b)

<class 'numpy.ndarray'>
-5.669342 2.2027073 -0.64129794


In [24]:
base = Image.new('RGBA', (img_w, img_h), color=(255, 255, 255))

In [12]:
anchors = list(zip(proj[-3:], dist[-3:]))

In [13]:
def translate(a, min_a, max_a, min_b, max_b):
    range_a = max_a - min_a
    range_b = max_b - min_b
    return min_b + ((a - min_a) * range_b / range_a)

In [25]:
draw = ImageDraw.Draw(base)
for coords, size, colorData in zip(sortedProj, sortedDist, sortedDim3):
    r, g, b = [int(translate(colorData[0], min_r, max_r, 0, 255)),
               int(translate(colorData[1], min_g, max_g, 0, 255)),
               int(translate(colorData[2], min_b, max_b, 0, 255))]
    x, y = coords
    x = translate(x, min_x, max_x, border_px, max_iw)
    y = translate(y, min_y, max_y, max_ih, border_px)
    size += 2
    bound = [(x-size, y-size), (x+size, y+size)]
    draw.ellipse(bound, fill=(r, g, b))


In [17]:
print(translate(max_x, min_x, max_x, max_iw, border_px))
print(max_iw)

100.0
1900


In [26]:
for coords in proj[-3:]:
    x, y = coords
    fill_c = (255, 255, 255)
    ol_c = (0, 0, 0)
    size = 30
    x = translate(x, min_x, max_x, border_px, max_iw)
    y = translate(y, min_y, max_y, max_ih, border_px)
    bound = [(x-size, y-size), (x+size, y+size)]
    draw.ellipse(bound, outline=ol_c, width=6)

base.show()

In [27]:
base.save('image.png')

In [199]:
min(sortedDist)

0.0